In [3]:
import numpy as np
import sklearn
from scipy import spatial
from statistics import mode
import sklearn.metrics.pairwise
from statistics import mean
import random
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import multivariate_normal

In [17]:
#Need to read in the data here
with open('2gaussian.txt') as gauss_file:
  all_lines = gauss_file.readlines()
  total_data = []
#print(all_lines)
for line in all_lines:
  line_list = line.strip('\n').split(' ')
  new_list = []
  for i in line_list:
    new_list.append(float(i))
  total_data.append(new_list)

In [28]:
#Generate random initial values
k = 2
in_cluster = random.sample(range(0,6000),k)
total_data = np.array(total_data)
#Those become the centroids of the two points
cluster_means = []
for i in range(len(in_cluster)):
  cluster_means.append(total_data[in_cluster[i]])
cluster_means = np.array(cluster_means)
mean1 = cluster_means[0]
mean2 = cluster_means[1]


#Calculate covariance of all assignments (Needs to change in the while loop), (# of gaussian mixtures times the dimensions fo the data set (shape will be 2,2,2))
cov_data1 = []
cov_data2 = []
for i in range(len(total_data)):
  cov1 = np.cov(total_data[i], cluster_means[0])
  cov2 = np.cov(total_data[i], cluster_means[1])
  cov_data1.append(cov1)
  cov_data2.append(cov2)


prob_list = []
for i in range(6000):
  prob1 = random.uniform(0,1)
  prob2 = 1-prob1
  probs = []
  probs.append(prob1)
  probs.append(prob2)
  prob_list.append(probs)

prob_list = np.array(prob_list)
covk1 = sum(cov_data1*prob_list[0])/sum(prob_list[0])
covk2 = sum(cov_data2*prob_list[1])/sum(prob_list[1])

#Calculate weights (pick two probabilities that add up to 1)
weights = [.5,.5]

#Begin while loop here

iter = 0

while(iter <10):

#E Step
#Calculate probabilities (p(x1|k)
  model1 = multivariate_normal(mean1, covk1, allow_singular = True)
  model2 = multivariate_normal(mean2, covk2, allow_singular = True)
  prob_list = []
  for i in range(len(total_data)):
     prob1 = model1.pdf(total_data[i])
     prob2 = model2.pdf(total_data[i])
     probs = []
     probs.append(prob1)
     probs.append(prob2)
     prob_list.append(probs)
  prob_list = np.array(prob_list)

   #Calculate pik
   #x times probabilities (All elements)/sum of prob in the cluster]
   #Need to normalize the piks
  pik = []
  for i in range(len(total_data)):
    piks = []
    new_pik1 = (prob_list[i][0]* weights[0])
    new_pik2 = (prob_list[i][1]* weights[1])
    piks.append(new_pik1)
    piks.append(new_pik2)
    pik.append(piks)
  #normalization of piks
  pik = np.array(pik)
  pik_sum = np.sum(pik,axis = 1)
  for i in range(len(pik)):
    pik[i] = pik[i]/pik_sum[i]

  old_mean1 = mean1
  old_mean2 = mean2
  #Mstep
  #Means
  mean1_data = []
  mean2_data = []
  pik1_sum = sum(pik[0])
  pik2_sum = sum(pik[1])
  for i in range(len(total_data)):
    mean1 = pik[i][0] * total_data[i]
    mean2 = pik[i][1] * total_data[i]
    mean1_data.append(mean1)
    mean2_data.append(mean2)
  mean1 = sum(mean1_data)/np.sum(pik, axis = 0)
  mean2 = sum(mean2_data)/np.sum(pik, axis = 0)


  if np.array_equal(old_mean1,mean1) and np.array_equal(old_mean2,mean2):
    break

  #covariance
  cov_data1 = []
  cov_data2 = []
  for i in range(k):
    diff1 = (total_data - mean1).T
    diff2 = (total_data-mean2).T
    new_pik_sum = np.sum(pik, axis = 0)
    w1_sum = np.dot(pik[:,0]*diff1,diff1.T)
    covk1 = w1_sum/new_pik_sum[0]
    w2_sum = np.dot(pik[:,1]*diff2,diff2.T)
    covk2 = w2_sum/new_pik_sum[1]



  #Weights
  weights = []
  w1 = 0
  w2 = 0
  for i in range(len(pik)):
    w1 += pik[i][0]
    w2 += pik[i][1]
  weights.append(w1/6000)
  weights.append(w2/6000)
  
  iter+=1

In [29]:
print('means1',mean1)
print('means2',mean2)

means1 [7.19246972 5.42523813]
means2 [2.82509076 3.02971859]


## 3 Gaussian Data- GMM Implementation

In [4]:
#Need to read in the data here
with open('3gaussian.txt') as gauss_file:
  all_lines = gauss_file.readlines()
  total_data = []
#print(all_lines)
for line in all_lines:
  line_list = line.strip('\n').split(' ')
  new_list = []
  for i in line_list:
    new_list.append(float(i))
  total_data.append(new_list)

In [ ]:
#Generate random initial values
k = 3
in_cluster = random.sample(range(0,6000),k)
total_data = np.array(total_data)
#Those become the centroids of the two points
cluster_means = []
for i in range(len(in_cluster)):
  cluster_means.append(total_data[in_cluster[i]])
cluster_means = np.array(cluster_means)
mean1 = cluster_means[0]
mean2 = cluster_means[1]
mean3 = cluster_means[2]


#Calculate covariance of all assignments (Needs to change in the while loop), (# of gaussian mixtures times the dimensions fo the data set (shape will be 2,2,2))
cov_data1 = []
cov_data2 = []
cov_data3 = []
for i in range(len(total_data)):
  cov1 = np.cov(total_data[i], cluster_means[0])
  cov2 = np.cov(total_data[i], cluster_means[1])
  cov3 = np.cov(total_data[i],cluster_means[2])
  cov_data1.append(cov1)
  cov_data2.append(cov2)
  cov_data3.append(cov3)


prob_list = []
for i in range(10000):
  prob1 = random.uniform(0,1)
  prob2 = (1-prob1)/2
  prob3 = prob2
  probs = []
  probs.append(prob1)
  probs.append(prob2)
  probs.append(prob3)
  prob_list.append(probs)

prob_list = np.array(prob_list)
covk1 = sum(cov_data1*prob_list[0][0:1])/sum(prob_list[0])
covk2 = sum(cov_data2*prob_list[1][0:1])/sum(prob_list[1])
covk3 = sum(cov_data3*prob_list[2][0:1])/sum(prob_list[2])

#Calculate weights (pick two probabilities that add up to 1)
weights = [1/3,1/3,1/3]

#Begin while loop here

iter = 0

while(True):

#E Step
#Calculate probabilities (p(x1|k)
  model1 = multivariate_normal(mean1, covk1, allow_singular = True)
  model2 = multivariate_normal(mean2, covk2, allow_singular = True)
  model3 = multivariate_normal(mean3, covk3, allow_singular = True)
  prob_list = []
  for i in range(len(total_data)):
     prob1 = model1.pdf(total_data[i])
     prob2 = model2.pdf(total_data[i])
     prob3 = model3.pdf(total_data[i])
     probs = []
     probs.append(prob1)
     probs.append(prob2)
     probs.append(prob3)
     prob_list.append(probs)
  prob_list = np.array(prob_list)

   #Calculate pik
   #x times probabilities (All elements)/sum of prob in the cluster]
   #Need to normalize the piks
  pik = []
  for i in range(len(total_data)):
    piks = []
    new_pik1 = (prob_list[i][0]* weights[0])
    new_pik2 = (prob_list[i][1]* weights[1])
    new_pik3 = (prob_list[i][2]* weights[2])
    piks.append(new_pik1)
    piks.append(new_pik2)
    piks.append(new_pik3)
    pik.append(piks)
  #normalization of piks
  pik = np.array(pik)
  pik_sum = np.sum(pik,axis = 1)
  for i in range(len(pik)):
    pik[i] = pik[i]/pik_sum[i]

  old_mean1 = mean1
  old_mean2 = mean2
  old_mean3 = mean3
  #Mstep
  #Means
  mean1_data = []
  mean2_data = []
  mean3_data = []
  pik1_sum = sum(pik[0])
  pik2_sum = sum(pik[1])
  pik3_sum = sum(pik[2])
  for i in range(len(total_data)):
    mean1 = pik[i][0] * total_data[i]
    mean2 = pik[i][1] * total_data[i]
    mean3 = pik[i][2] * total_data[i]
    mean1_data.append(mean1)
    mean2_data.append(mean2)
    mean3_data.append(mean3)
  mean1 = sum(mean1_data)/np.sum(pik, axis = 0)[0]
  mean2 = sum(mean2_data)/np.sum(pik, axis = 0)[1]
  mean3 = sum(mean3_data)/np.sum(pik, axis = 0)[2]


  if np.array_equal(old_mean1,mean1) and np.array_equal(old_mean2,mean2) and np.array_equal(old_mean3,mean3):
    break

  #covariance
  cov_data1 = []
  cov_data2 = []
  cov_data3
  for i in range(k):
    diff1 = (total_data - mean1).T
    diff2 = (total_data-mean2).T
    diff3 = (total_data-mean3).T
    new_pik_sum = np.sum(pik, axis = 0)
    w1_sum = np.dot(pik[:,0]*diff1,diff1.T)
    covk1 = w1_sum/new_pik_sum[0]
    w2_sum = np.dot(pik[:,1]*diff2,diff2.T)
    covk2 = w2_sum/new_pik_sum[1]
    w3_sum = np.dot(pik[:,2]*diff3,diff3.T)
    covk3 = w3_sum/new_pik_sum[2]



  #Weights
  weights = []
  w1 = 0
  w2 = 0
  w3 = 0
  for i in range(len(pik)):
    w1 += pik[i][0]
    w2 += pik[i][1]
    w3 += pik[i][2]
  weights.append(w1/10000)
  weights.append(w2/10000)
  weights.append(w3/10000)
  
  iter+=1

In [16]:
print('mean1',mean1)
print('mean2',mean2)
print('mean3',mean3)

mean1 [3.03975701 3.04866909]
mean2 [5.01175547 7.00149623]
mean3 [7.02157607 4.01546857]
